#### 归一化 normalization

##### 归一化的目的

MSE损失函数是凸函数

<img src="./Img/10.png" width="40%" height="40%">   

如果样本的值X1 << X2，那么最终θ1 >> θ2

θ1和θ2数值在一开始初始化是差不多的，但是由于X1和X2的关系，我们会发现θ1从初始值到目标位置的距离要远大于θ2从初始值到目标位置，这也是第二幅图扁平的原因

每次调整θ1的幅度要远小于θ2的幅度

总结：归一化的目的是使得最终梯度下降的时候可以不同维度θ参数可以在接近的调整幅度上。

扩展：归一化可以提高精度，主要在距离计算上，样本之间的数量级不同会导致偏向性，所以归一化后就没这个问题了

##### 最大值最小值归一化

min max scaling，针对每一列，每一列取出最大最小值，然后对这一列中每个数值按公式操作

$ X_{\text{norm}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}} $

* $X$ 是原始数据点。
* $X_{min}$ 是数据集中的最小值。
* $X_{max}$ 是数据集中的最大值。
* $X_{norm}$ 是归一化后的数据点。


优点是一定可以把数值归一到0到1之间，缺点是如果有一个离群值，影响会比较大